In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b1444cf62af39aa43c313f954bf02e46748e65e29e1918b25dcd330bab71792a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import regexp_extract, regexp_replace, split, round, col, format_number, array_contains
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression




In [ ]:
spark = SparkSession.builder.appName("PySparkJob").getOrCreate()

In [ ]:
phones = spark.read.format("csv").option("header", "true").load("train.csv")

In [ ]:
phones.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [ ]:
phones.columns

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [ ]:
phones.schema

StructType([StructField('battery_power', StringType(), True), StructField('blue', StringType(), True), StructField('clock_speed', StringType(), True), StructField('dual_sim', StringType(), True), StructField('fc', StringType(), True), StructField('four_g', StringType(), True), StructField('int_memory', StringType(), True), StructField('m_dep', StringType(), True), StructField('mobile_wt', StringType(), True), StructField('n_cores', StringType(), True), StructField('pc', StringType(), True), StructField('px_height', StringType(), True), StructField('px_width', StringType(), True), StructField('ram', StringType(), True), StructField('sc_h', StringType(), True), StructField('sc_w', StringType(), True), StructField('talk_time', StringType(), True), StructField('three_g', StringType(), True), StructField('touch_screen', StringType(), True), StructField('wifi', StringType(), True), StructField('price_range', StringType(), True)])

In [ ]:
for col_name in phones.columns:
  if col_name in ['clock_speed', 'm_dep']:
    phones = phones.withColumn(col_name, phones[col_name].cast(FloatType()))
  else:
    phones = phones.withColumn(col_name, phones[col_name].cast(IntegerType()))

In [ ]:
phones.schema

StructType([StructField('battery_power', IntegerType(), True), StructField('blue', IntegerType(), True), StructField('clock_speed', FloatType(), True), StructField('dual_sim', IntegerType(), True), StructField('fc', IntegerType(), True), StructField('four_g', IntegerType(), True), StructField('int_memory', IntegerType(), True), StructField('m_dep', FloatType(), True), StructField('mobile_wt', IntegerType(), True), StructField('n_cores', IntegerType(), True), StructField('pc', IntegerType(), True), StructField('px_height', IntegerType(), True), StructField('px_width', IntegerType(), True), StructField('ram', IntegerType(), True), StructField('sc_h', IntegerType(), True), StructField('sc_w', IntegerType(), True), StructField('talk_time', IntegerType(), True), StructField('three_g', IntegerType(), True), StructField('touch_screen', IntegerType(), True), StructField('wifi', IntegerType(), True), StructField('price_range', IntegerType(), True)])

In [ ]:
phones.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [ ]:
feature_columns = ['battery_power','blue','clock_speed','dual_sim','fc','four_g',\
                   'int_memory','m_dep','mobile_wt','n_cores','pc','px_height','px_width', \
                   'ram','sc_h','sc_w','talk_time','three_g','touch_screen','wifi']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [ ]:
phones = phones.na.drop("any")

In [ ]:
train_df, test_df, test2_df = phones.randomSplit([0.5, 0.3, 0.2], seed=42)

In [ ]:
test_df.schema

StructType([StructField('battery_power', IntegerType(), True), StructField('blue', IntegerType(), True), StructField('clock_speed', FloatType(), True), StructField('dual_sim', IntegerType(), True), StructField('fc', IntegerType(), True), StructField('four_g', IntegerType(), True), StructField('int_memory', IntegerType(), True), StructField('m_dep', FloatType(), True), StructField('mobile_wt', IntegerType(), True), StructField('n_cores', IntegerType(), True), StructField('pc', IntegerType(), True), StructField('px_height', IntegerType(), True), StructField('px_width', IntegerType(), True), StructField('ram', IntegerType(), True), StructField('sc_h', IntegerType(), True), StructField('sc_w', IntegerType(), True), StructField('talk_time', IntegerType(), True), StructField('three_g', IntegerType(), True), StructField('touch_screen', IntegerType(), True), StructField('wifi', IntegerType(), True), StructField('price_range', IntegerType(), True)])

# RandomForestClassifier


In [ ]:
rf = RandomForestClassifier(featuresCol="features", labelCol="price_range")
pipeline = Pipeline(stages=[assembler, rf])
rf_model = pipeline.fit(train_df)

In [ ]:
rf_prediction = rf_model.transform(test_df)
rf_prediction.select("prediction", "price_range", "features").show()

+----------+-----------+--------------------+
|prediction|price_range|            features|
+----------+-----------+--------------------+
|       2.0|          1|[501.0,0.0,2.2999...|
|       2.0|          2|[501.0,1.0,0.5,1....|
|       2.0|          1|[503.0,0.0,1.2000...|
|       0.0|          0|[503.0,0.0,2.5,0....|
|       0.0|          0|[504.0,1.0,0.5,1....|
|       2.0|          2|[504.0,1.0,2.7999...|
|       1.0|          1|[507.0,1.0,1.8999...|
|       0.0|          0|[508.0,0.0,0.8000...|
|       1.0|          1|[508.0,1.0,1.2999...|
|       3.0|          3|[510.0,1.0,2.0,1....|
|       3.0|          2|[511.0,0.0,0.6999...|
|       0.0|          0|[511.0,1.0,0.6000...|
|       0.0|          0|[513.0,1.0,0.5,0....|
|       3.0|          3|[514.0,1.0,1.6000...|
|       0.0|          0|[516.0,0.0,1.1000...|
|       3.0|          2|[516.0,1.0,0.6999...|
|       3.0|          3|[517.0,0.0,1.3999...|
|       0.0|          0|[523.0,0.0,2.5999...|
|       0.0|          0|[527.0,0.0

In [ ]:
evaluator_accuracy = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="price_range", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(rf_prediction)
print("Accuracy:", accuracy)

# Для F1-score (F1-меры)
evaluator_f1 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="price_range", metricName="f1")
f1_score = evaluator_f1.evaluate(rf_prediction)
print("F1-score:", f1_score)

Accuracy: 0.8172920065252854
F1-score: 0.8146087514458515


# Логистическая регрессия

In [ ]:
lr  = LogisticRegression(featuresCol="features", labelCol="price_range")
pipeline = Pipeline(stages=[assembler, lr])
lr_model = pipeline.fit(train_df)

In [ ]:
lr_prediction = lr_model.transform(test_df)
lr_prediction.select("prediction", "price_range", "features").show()

+----------+-----------+--------------------+
|prediction|price_range|            features|
+----------+-----------+--------------------+
|       1.0|          1|[501.0,0.0,2.2999...|
|       2.0|          2|[501.0,1.0,0.5,1....|
|       1.0|          1|[503.0,0.0,1.2000...|
|       0.0|          0|[503.0,0.0,2.5,0....|
|       0.0|          0|[504.0,1.0,0.5,1....|
|       2.0|          2|[504.0,1.0,2.7999...|
|       1.0|          1|[507.0,1.0,1.8999...|
|       0.0|          0|[508.0,0.0,0.8000...|
|       1.0|          1|[508.0,1.0,1.2999...|
|       3.0|          3|[510.0,1.0,2.0,1....|
|       2.0|          2|[511.0,0.0,0.6999...|
|       0.0|          0|[511.0,1.0,0.6000...|
|       0.0|          0|[513.0,1.0,0.5,0....|
|       3.0|          3|[514.0,1.0,1.6000...|
|       0.0|          0|[516.0,0.0,1.1000...|
|       2.0|          2|[516.0,1.0,0.6999...|
|       3.0|          3|[517.0,0.0,1.3999...|
|       0.0|          0|[523.0,0.0,2.5999...|
|       0.0|          0|[527.0,0.0

In [ ]:
evaluator_accuracy = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="price_range", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(lr_prediction)
print("Accuracy", accuracy)

Accuracy 0.9608482871125612


In [ ]:
# Оценка F1-меры
evaluator_f1 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="price_range", metricName="f1")
f1_score = evaluator_f1.evaluate(lr_prediction)
print("F1-score:", f1_score)

F1-score: 0.9607421237811605


In [ ]:
lr_model.write().overwrite().save('lr_model')

In [ ]:
!zip -r /content/lr_model.zip lr_model

  adding: lr_model/ (stored 0%)
  adding: lr_model/metadata/ (stored 0%)
  adding: lr_model/metadata/part-00000 (deflated 22%)
  adding: lr_model/metadata/_SUCCESS (stored 0%)
  adding: lr_model/metadata/._SUCCESS.crc (stored 0%)
  adding: lr_model/metadata/.part-00000.crc (stored 0%)
  adding: lr_model/stages/ (stored 0%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/ (stored 0%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/metadata/ (stored 0%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/metadata/part-00000 (deflated 39%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/metadata/_SUCCESS (stored 0%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/metadata/._SUCCESS.crc (stored 0%)
  adding: lr_model/stages/0_VectorAssembler_9aefcf367bab/metadata/.part-00000.crc (stored 0%)
  adding: lr_model/stages/1_LogisticRegression_1aa3c8130e15/ (stored 0%)
  adding: lr_model/stages/1_LogisticRegression_1aa3c8130e15/data/ (stored 0%)
  adding: 